In [33]:
import os 
import glob 
from tqdm import tqdm

import torch
import pandas as pd
import numpy as np
import skimage.metrics 
from torchmetrics.functional.classification import dice, recall, precision
from torchmetrics import Dice, Recall, Precision 
import nibabel as nib 

In [7]:
def binarize_image(img, threshold = 0.5, one_hot = False):
    if img.ndim == 4:
       img = img.unsqueeze(0)

    elif img.ndim == 3:
        img = img[None, None, :, :, :]

    assert img.ndim == 5, f'Binarize_image, tensor mismatch {img.shape}'

    n_channels = img.shape[1]

    # binary problem
    if n_channels == 1:
        nimg = img > threshold
    elif n_channels == 3:
        if img.dtype == torch.bool:
            nimg = img.float()
        else:
            nimg           = torch.zeros_like(img)
            argmax_indexes = torch.argmax(img, dim = 1)
            nimg.scatter_(1, argmax_indexes.unsqueeze(1), 1) 
    else:
        print(f"In binarize_image, number of channels {n_channels}")
    
    if nimg.dtype != torch.float:   nimg = nimg.float()
    
    return nimg

def calculate_overlap_metrics(pred, gt, target_label: int == 1):
    aneur_mask      = torch.where(gt == target_label, 1, 0)
    pred_image_bin  = binarize_image(pred)
    pred_aneur_mask = torch.mul(pred_image_bin, aneur_mask)

    # compute dice score recall and precision
    tp = torch.sum((pred_aneur_mask == 1) & (aneur_mask == 1))
    fp = torch.sum((pred_aneur_mask == 1) & (aneur_mask == 0))
    fn = torch.sum((pred_aneur_mask == 0) & (aneur_mask == 1))

    if 2*tp + fp + fn == 0: dice_aneur = 1e-12
    else: dice_aneur = (2*tp/(2*tp + fp + fn)).item()

    if tp + fn == 0: recall_aneur = 1e-12
    else: recall_aneur = (tp/(tp+fn)).item()

    if fp + tp == 0: precision_aneur = 1e-12
    else: precision_aneur = (tp/(tp+fp)).item()
    metrics = {'dice_aneur':dice_aneur, 
               'recall_aneur':recall_aneur, 
               'precision_aneur':precision_aneur}
    
    return metrics

In [8]:
def measure_other_metrics(
        gt_vols_fp: list,
        pred_vols_fn: list,
        predictions_dir: str,
        metrics_tm: dict
) -> pd.DataFrame:
    results = []
    for gt_vol_fp in tqdm(gt_vols_fp):
        vol_fn = os.path.basename(gt_vol_fp)
    
        assert vol_fn in pred_vols_fn, \
            f"No prediction for vol {vol_fn}"
        
        # load vols     
        gt = nib.load(gt_vol_fp).get_fdata()
        gt = torch.tensor(gt).float()
        
        pred = nib.load(os.path.join(predictions_dir, vol_fn)).get_fdata()
        pred = torch.tensor(pred).float()
    
        # Calculate metrics
        metrics = calculate_overlap_metrics(pred, gt, target_label= 1)
        metrics = {f'{k}_kostas':v for k,v in metrics.items()}
        
        for metric_name, metric_tm in metrics_tm.items():
            metrics[f'{metric_name}_tm'] = metric_tm(pred.int(), gt.int()).item()
        
        metrics['mhd'] = skimage.metrics.hausdorff_distance(gt.cpu().numpy(), pred.cpu().numpy(), method='modified')
        
        # Add metrics to the results list
        results.append({
            'vol_name': vol_fn,
            **metrics
        })
    
    return pd.DataFrame(results)
    

# Evaluate trained on source domain and predicting in holdout set of the same domain

Keep Dice-Score, Recall, Precision, and modified Hausdorff Distance for each volume and each class


## UZS

### Binary Segmentation All vessels

In [9]:
data_dir = '../../../../data/'
ground_truth_dir = os.path.join(data_dir, 'preprocessed/Mathijs/Dataset001_BinaryAllVessels/labelsTs_binary_all_classes')
predictions_dir = os.path.join(data_dir, 'nnUNet_predictions', 'train_on_SD_predict_on_SD', 'Dataset001_BinaryAllVessels', '3d_fullres')

In [28]:
gt_vols_fp = glob.glob(os.path.join(ground_truth_dir, '*.nii.gz'))
pred_vols_fn = os.listdir(predictions_dir)

In [29]:
pred_vols_fn

['18.nii.gz',
 'predict_from_raw_data_args.json',
 '19.nii.gz',
 '61.nii.gz',
 '49.nii.gz',
 '31.nii.gz',
 '51.nii.gz',
 '48.nii.gz',
 '27.nii.gz',
 'dataset.json',
 '1.nii.gz',
 '0.nii.gz',
 '3.nii.gz',
 '7.nii.gz',
 'plans.json',
 '25.nii.gz']

In [84]:
dice = Dice(num_classes=2, ignore_index=0, average='micro')
recall = Recall(num_classes=2, ignore_index=0, average='micro', task='binary')
precision = Precision(num_classes=2, ignore_index=0, average='micro', task='binary')

metrics_tm = {'dice':dice, 'recall':recall, 'precision':precision}

In [85]:
results = []
for gt_vol_fp in tqdm(gt_vols_fp):
    vol_fn = os.path.basename(gt_vol_fp)
    
    assert vol_fn in pred_vols_fn, \
        f"No prediction for vol {vol_fn}"
    
    # load vols     
    gt = nib.load(gt_vol_fp).get_fdata()
    gt = torch.tensor(gt).float()
    
    pred = nib.load(os.path.join(predictions_dir, vol_fn)).get_fdata()
    pred = torch.tensor(pred).float()

    # Calculate metrics
    metrics = calculate_overlap_metrics(pred, gt, target_label= 1)
    metrics = {f'{k}_kostas':v for k,v in metrics.items()}
    
    for metric_name, metric_tm in metrics_tm.items():
        metrics[f'{metric_name}_tm'] = metric_tm(pred.int(), gt.int()).item()
    
    metrics['mhd'] = skimage.metrics.hausdorff_distance(gt.cpu().numpy(), pred.cpu().numpy(), method='modified')
    
    # Add metrics to the results list
    results.append({
        'vol_name': vol_fn,
        **metrics
    })

100%|██████████| 13/13 [10:39<00:00, 49.19s/it]


In [93]:
results_df = pd.DataFrame(results)
results_df

,vol_name,dice_aneur_kostas,recall_aneur_kostas,precision_aneur_kostas,dice_tm,recall_tm,precision_tm,mhd
0,18.nii.gz,tensor(0.9534),tensor(0.9110),tensor(1.),0.864702,0.910991,1.0,0.749403
1,19.nii.gz,tensor(0.9368),tensor(0.8812),tensor(1.),0.862072,0.881189,1.0,2.720930
2,61.nii.gz,tensor(0.9096),tensor(0.8342),tensor(1.),0.862584,0.834175,1.0,2.124458
3,49.nii.gz,tensor(0.9271),tensor(0.8641),tensor(1.),0.898427,0.864064,1.0,1.139996
4,31.nii.gz,tensor(0.9391),tensor(0.8851),tensor(1.),0.883650,0.885113,1.0,0.832233
5,51.nii.gz,tensor(0.9202),tensor(0.8521),tensor(1.),0.889157,0.852123,1.0,0.787224
6,48.nii.gz,tensor(0.9627),tensor(0.9281),tensor(1.),0.782365,0.928078,1.0,10.144669
7,27.nii.gz,tensor(0.9067),tensor(0.8293),tensor(1.),0.841543,0.829288,1.0,1.210820
8,1.nii.gz,tensor(0.8759),tensor(0.7792),tensor(1.),0.850352,0.779167,1.0,1.464861
9,0.nii.gz,tensor(0.8925),tensor(0.8058),tensor(1.),0.813883,0.805802,1.0,2.892901


In [94]:
os.makedirs(os.path.join(data_dir, 'results'), exist_ok=True)

In [95]:
results_df.to_csv(os.path.join(data_dir, 'results', 'sd_usz__td_usz__binary_all_vessels.csv'), index=False)

In [96]:
results_df.dice_tm.mean()

0.8578855532866257

In [97]:
results_df.mhd.mean()


2.07031823880332

### Binary Segmentation Aneurysm vs Background

In [105]:
data_dir = '../../../../data/'
ground_truth_dir = os.path.join(data_dir, 'preprocessed/Mathijs/Dataset002_BinaryAneurysmOnly/labelsTs_binary_only_aneurysm')
predictions_dir = os.path.join(data_dir, 'nnUNet_predictions', 'train_on_SD_predict_on_SD', 'Dataset002_BinaryAneurysmOnly', '3d_fullres')

In [106]:
gt_vols_fp = glob.glob(os.path.join(ground_truth_dir, '*.nii.gz'))
pred_vols_fn = os.listdir(predictions_dir)

In [107]:
dice = Dice(num_classes=2, ignore_index=0, average='micro')
recall = Recall(num_classes=2, ignore_index=0, average='micro', task='binary')
precision = Precision(num_classes=2, ignore_index=0, average='micro', task='binary')

metrics_tm = {'dice':dice, 'recall':recall, 'precision':precision}

In [110]:
results = []
for gt_vol_fp in tqdm(gt_vols_fp):
    vol_fn = os.path.basename(gt_vol_fp)

    assert vol_fn in pred_vols_fn, \
        f"No prediction for vol {vol_fn}"
    
    # load vols     
    gt = nib.load(gt_vol_fp).get_fdata()
    gt = torch.tensor(gt).float()
    
    pred = nib.load(os.path.join(predictions_dir, vol_fn)).get_fdata()
    pred = torch.tensor(pred).float()

    # Calculate metrics
    metrics = calculate_overlap_metrics(pred, gt, target_label= 1)
    metrics = {f'{k}_kostas':v for k,v in metrics.items()}
    
    for metric_name, metric_tm in metrics_tm.items():
        metrics[f'{metric_name}_tm'] = metric_tm(pred.int(), gt.int()).item()
    
    metrics['mhd'] = skimage.metrics.hausdorff_distance(gt.cpu().numpy(), pred.cpu().numpy(), method='modified')
    
    # Add metrics to the results list
    results.append({
        'vol_name': vol_fn,
        **metrics
    })

100%|██████████| 13/13 [08:16<00:00, 38.21s/it]


In [111]:
results_df = pd.DataFrame(results)
results_df

,vol_name,dice_aneur_kostas,recall_aneur_kostas,precision_aneur_kostas,dice_tm,recall_tm,precision_tm,mhd
0,18.nii.gz,0.942420,0.891109,1.000000e+00,0.834174,0.891109,1.0,27.644806
1,19.nii.gz,0.797440,0.663118,1.000000e+00,0.781393,0.663118,1.0,1.145862
2,61.nii.gz,0.000000,0.000000,1.000000e-12,0.000000,0.000000,0.0,inf
3,49.nii.gz,0.844707,0.731162,1.000000e+00,0.831515,0.731162,1.0,2.806254
4,31.nii.gz,0.910970,0.836496,1.000000e+00,0.829733,0.836496,1.0,9.663954
5,51.nii.gz,0.672888,0.507032,1.000000e+00,0.496017,0.507032,1.0,15.943432
6,48.nii.gz,0.992823,0.985749,1.000000e+00,0.961910,0.985749,1.0,0.070813
7,27.nii.gz,0.870654,0.770936,1.000000e+00,0.803594,0.770936,1.0,0.742497
8,1.nii.gz,0.000000,0.000000,1.000000e-12,0.000000,0.000000,0.0,68.120221
9,0.nii.gz,0.870571,0.770807,1.000000e+00,0.858196,0.770807,1.0,1.183752


Note it could not identify aneurysms at all in 2 cases, scan 61 and scan 1. The rest of them have a very high value except for 51.

In [112]:
os.makedirs(os.path.join(data_dir, 'results'), exist_ok=True)

In [113]:
results_df.to_csv(os.path.join(data_dir, 'results', 'sd_usz__td_usz__binary_aneurysm_only.csv'), index=False)

In [114]:
results_df.dice_tm.mean()

0.6842775000975683

In [116]:
import numpy as np
results_df.mhd.map(lambda x: x if x != np.inf else 68).mean()


15.335989475071896

### 3 classes: Aneurysm, Vessels, Background

In [117]:
data_dir = '../../../../data/'
ground_truth_dir = os.path.join(data_dir, 'preprocessed/Mathijs/Dataset003_3Classes/labelsTs_3_classes')
predictions_dir = os.path.join(data_dir, 'nnUNet_predictions', 'train_on_SD_predict_on_SD', 'Dataset003_3Classes', '3d_fullres')

In [118]:
gt_vols_fp = glob.glob(os.path.join(ground_truth_dir, '*.nii.gz'))
pred_vols_fn = os.listdir(predictions_dir)

In [133]:
num_classes = 3
dice = Dice(num_classes=num_classes, ignore_index=0, average='micro')
recall = Recall(num_classes=num_classes, ignore_index=0, average='micro', task='multiclass')
precision = Precision(num_classes=num_classes, ignore_index=0, average='micro', task='multiclass')

metrics_tm = {'dice':dice, 'recall':recall, 'precision':precision}

In [164]:
results = []
for gt_vol_fp in tqdm(gt_vols_fp):
    vol_fn = os.path.basename(gt_vol_fp)

    assert vol_fn in pred_vols_fn, \
        f"No prediction for vol {vol_fn}"
    
    # load vols     
    gt = nib.load(gt_vol_fp).get_fdata()
    gt = torch.tensor(gt).float()
    
    pred = nib.load(os.path.join(predictions_dir, vol_fn)).get_fdata()
    pred = torch.tensor(pred).float()

    # Calculate metrics
    metrics = calculate_overlap_metrics(pred, gt, target_label=2)
    metrics = {f'{k}_kostas':v for k,v in metrics.items()}

    for metric_name, metric_tm in metrics_tm.items():
        metrics[f'{metric_name}_tm'] = metric_tm(pred.int(), gt.int()).item()

    metrics['mhd'] = skimage.metrics.hausdorff_distance(gt.cpu().numpy(), pred.cpu().numpy(), method='modified')

    # Calculate the aneurysm only metrics
    gt = torch.where(gt == 2, 1, 0)
    pred = torch.where(pred == 2, 1, 0)
    
    for metric_name, metric_tm in metrics_tm.items():
         metrics[f'{metric_name}_tm_aneurysm_only'] = metric_tm(pred.int(), gt.int()).item()
    metrics = dict()
    metrics['mhd_aneurysm_only'] = skimage.metrics.hausdorff_distance(gt.cpu().numpy(), pred.cpu().numpy(), method='modified')

    # Add metrics to the results list
    results.append({
        'vol_name': vol_fn,
        **metrics
    })

100%|██████████| 13/13 [04:41<00:00, 21.64s/it]


In [154]:
results_df = pd.DataFrame(results)

In [155]:
results_df.columns

Index(['vol_name', 'dice_aneur_kostas', 'recall_aneur_kostas',
       'precision_aneur_kostas', 'dice_tm', 'recall_tm', 'precision_tm', 'mhd',
       'dice_tm_aneurysm_only', 'recall_tm_aneurysm_only',
       'precision_tm_aneurysm_only'],
      dtype='object')

In [156]:
results_df

,vol_name,dice_aneur_kostas,recall_aneur_kostas,precision_aneur_kostas,dice_tm,recall_tm,precision_tm,mhd,dice_tm_aneurysm_only,recall_tm_aneurysm_only,precision_tm_aneurysm_only
0,18.nii.gz,0.992763,0.985631,1.0,0.843258,0.891947,0.891947,1.082557,0.853339,0.899416,0.899416
1,19.nii.gz,0.981017,0.962741,1.0,0.849466,0.891182,0.891182,2.336168,0.815594,0.730761,0.730761
2,61.nii.gz,0.931643,0.872033,1.0,0.847720,0.857916,0.857916,3.185457,0.388794,0.248418,0.248418
3,49.nii.gz,0.925006,0.860475,1.0,0.869198,0.856359,0.856359,3.399348,0.830266,0.750345,0.750345
4,31.nii.gz,0.962907,0.928467,1.0,0.878906,0.876689,0.876689,0.473720,0.879774,0.910706,0.910706
5,51.nii.gz,0.923935,0.858623,1.0,0.885509,0.900141,0.900141,0.459954,0.615766,0.552184,0.552184
6,48.nii.gz,0.995789,0.991613,1.0,0.742941,0.943133,0.943133,12.052795,0.974191,0.991361,0.991361
7,27.nii.gz,1.000000,1.000000,1.0,0.822212,0.863873,0.863873,2.849913,0.797040,0.928571,0.928571
8,1.nii.gz,0.865342,0.762646,1.0,0.789434,0.679630,0.679630,2.786238,0.200700,0.111543,0.111543
9,0.nii.gz,0.894966,0.809899,1.0,0.794191,0.795226,0.795226,3.435928,0.809009,0.707755,0.707755


In [170]:
results_df.to_csv(os.path.join(data_dir, 'results', 'sd_usz__td_usz__3classes_UIA_and_vessels.csv'), index=False)

In [160]:
results_df.dice_tm.mean()

0.8370189987696134

In [161]:
results_df.dice_tm_aneurysm_only.mean()

0.7600323248368043

In [163]:
import numpy as np
results_df.mhd.mean()#.map(lambda x: x if x != np.inf else 68).mean()

2.6824194654548474

In [171]:
results_df.mhd_aneurysm_only.mean()

6.136928550363725

### 21 classes

In [105]:
data_dir = '../../../../data/'
ground_truth_dir = os.path.join(data_dir, 'preprocessed/Mathijs/Dataset004_21Classes/labelsTs')
predictions_dir = os.path.join(data_dir, 'nnUNet_predictions', 'train_on_SD_predict_on_SD', 'Dataset004_21Classes', '3d_fullres')

In [106]:
gt_vols_fp = glob.glob(os.path.join(ground_truth_dir, '*.nii.gz'))
pred_vols_fn = os.listdir(predictions_dir)

In [107]:
dice = Dice(num_classes=21, ignore_index=0, average='micro')
recall = Recall(num_classes=21, ignore_index=0, average='micro', task='binary')
precision = Precision(num_classes=21, ignore_index=0, average='micro', task='binary')

metrics_tm = {'dice':dice, 'recall':recall, 'precision':precision}

In [110]:
results = []
for gt_vol_fp in tqdm(gt_vols_fp):
    vol_fn = os.path.basename(gt_vol_fp)

    assert vol_fn in pred_vols_fn, \
        f"No prediction for vol {vol_fn}"
    
    # load vols     
    gt = nib.load(gt_vol_fp).get_fdata()
    gt = torch.tensor(gt).float()
    
    pred = nib.load(os.path.join(predictions_dir, vol_fn)).get_fdata()
    pred = torch.tensor(pred).float()

    # Calculate metrics
    metrics = calculate_overlap_metrics(pred, gt, target_label= 1)
    metrics = {f'{k}_kostas':v for k,v in metrics.items()}
    
    for metric_name, metric_tm in metrics_tm.items():
        metrics[f'{metric_name}_tm'] = metric_tm(pred.int(), gt.int()).item()
    
    metrics['mhd'] = skimage.metrics.hausdorff_distance(gt.cpu().numpy(), pred.cpu().numpy(), method='modified')
    
    # Add metrics to the results list
    results.append({
        'vol_name': vol_fn,
        **metrics
    })

100%|██████████| 13/13 [08:16<00:00, 38.21s/it]


In [111]:
results_df = pd.DataFrame(results)
results_df

,vol_name,dice_aneur_kostas,recall_aneur_kostas,precision_aneur_kostas,dice_tm,recall_tm,precision_tm,mhd
0,18.nii.gz,0.942420,0.891109,1.000000e+00,0.834174,0.891109,1.0,27.644806
1,19.nii.gz,0.797440,0.663118,1.000000e+00,0.781393,0.663118,1.0,1.145862
2,61.nii.gz,0.000000,0.000000,1.000000e-12,0.000000,0.000000,0.0,inf
3,49.nii.gz,0.844707,0.731162,1.000000e+00,0.831515,0.731162,1.0,2.806254
4,31.nii.gz,0.910970,0.836496,1.000000e+00,0.829733,0.836496,1.0,9.663954
5,51.nii.gz,0.672888,0.507032,1.000000e+00,0.496017,0.507032,1.0,15.943432
6,48.nii.gz,0.992823,0.985749,1.000000e+00,0.961910,0.985749,1.0,0.070813
7,27.nii.gz,0.870654,0.770936,1.000000e+00,0.803594,0.770936,1.0,0.742497
8,1.nii.gz,0.000000,0.000000,1.000000e-12,0.000000,0.000000,0.0,68.120221
9,0.nii.gz,0.870571,0.770807,1.000000e+00,0.858196,0.770807,1.0,1.183752


Note it could not identify aneurysms at all in 2 cases, scan 61 and scan 1. The rest of them have a very high value except for 51.

In [112]:
os.makedirs(os.path.join(data_dir, 'results'), exist_ok=True)

In [113]:
results_df.to_csv(os.path.join(data_dir, 'results', 'sd_usz__td_usz__binary_aneurysm_only.csv'), index=False)

In [114]:
results_df.dice_tm.mean()

0.6842775000975683